**NOTA**: Las respuestas no necesariamente deberian ser literales

# Minitarea 2

Nombre:

---------------------------
## Language Models (3 pts)
Estas preguntas son teóricas y deben ser resueltas con desarrollo, pero sin código. Por favor usen $\LaTeX$ para las fórmulas. El material del curso y los videos deberian ser suficientes para que puedan responder.


### Pregunta 1 (1 pt)
Asuma que tiene un modelo de lenguaje de trigramas (simple) entrenado sobre un corpus C. Muestre cómo el modelo le asigna una probabilidad a la oración `el perro le ladra al gato` usando los parámetros estimados a partir del corpus  $q(w_i | w_{i-2}, w_{i-1})$ .

**Respuesta:**

$ 
\begin{equation}
\begin{split}
P(el\: perro\: le\: ladra\: al\: gato\: STOP) & = q(el|*, *) \\
                                                & \times q(perro|*, el)\\
                                                & \times q(le|el, perro)\\
                                                & \times q(ladra|perro, le)\\
                                                & \times q(al|le, ladra) \\
                                                & \times q(gato|ladra, al)\\
                                                & \times q(STOP|al, gato)
\end{split}
\end{equation}
$

### Pregunta 2 (1 pt)
Muestre cómo se calcularía  $q(w_{i} | w_{i-2}, w_{i-1})$  usando un modelo que interpola un modelo de lenguajes de trigramas, bigramas, y unigrama.


**Respuesta:**

$
\begin{split}
q(w_{i} | w_{i-2}, w_{i-1}) & = \lambda_1 \times q_{ML}(w_{i} | w_{i-2}, w_{i-1})\\
                            & + \lambda_2 \times q_{ML}(w_{i} | w_{i-1}) \\
                            & + \lambda_3 \times q_{ML}(w_i)
\end{split}
$

Donde $\lambda_1$, $\lambda_2$ y $\lambda_3$ son parametros calculados usando un set de validacion.

### Pregunta 3 (1 pt)
¿Qué ventajas tiene el modelo interpolado sobre el modelo de trigramas simple?

**Respuesta:** (esta respuesta es mas abierta)

idea: los trigramas son muy escasos en un corpus de training porque hay demasiadas posibilidades, entonces muchos trigramas no aparecen en el corpus de entrenamiento y se vuelven 0 para estimaciones futuras. Por esto, se usan estimadores de maxima verosimilitud condicionados por menos contexto (o ninguno). Estos estimadores son mas robustos, en el sentido de que es menos probable que las ocurrecians de bigramas o unigramas sean cero en un corpus razonable, pero se ignora la informacion provista por el contexto para realizar la estimacion.

Al usar un modelo interpolado, se obtienen ventajas de ambas aproximaciones, el modelo es capaz de condicionar con respecto al contexto, pero la probabilidad no se vuelve 0 cuando el trigrama no aparecio en el set de entrenamiento.


-----------------------
## Naive Bayes (3 pts)
En esta parte de la minitarea deberan programar Naive Bayes Multinomial usando Laplace Smothing. Las referencias las pueden encontrar en el material del curso y los videos del profesor.

A continuacion se presentan un conjunto de documentos de training divididos en 2 categorias distintas. Ustedes deberan clasificar los documentos del test set usando Naive Bayes con Laplace Smothing.

Por favor, documenten su codigo. Escriban que hacen las funciones, que reciben, que entregan, etc. Si en algun lugar de su codigo hacen algo que creen que debe ser explicado, pongan comentarios, son bienvenidos. 


**Underflow prevention:** En vez de hacer muchas multiplicacions de `float`s, reemplacenlas por sumas de logaritmos para prevenir errores de precision. Revisen la diapo 69 de las slides. 

NOTA: Sobre las `namedtuple`s. Es el tipo de los documentos. Son objetos inmutable que tienen dos atributos: `words` donde estan las palabras del documento y `class_` donde se guarda la clase de ese documento. Estos objetos son inmutables, lo que quiere decir que si quieren modificar un documento y cambiarle la clase, tienen que crear otro documento. Otra cosa es que son tuplas como cualquier otra, es decir se pueden acceder usando indices como `doc[0]` o `doc[1]`. Son libres de implementar su solucion como quieran, si no les gusta este tipo de dato usen otro.



In [2]:
from collections import namedtuple
from pprint import pprint


document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

train_set = (
    document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
    document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
    document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
    document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
    document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
    document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
    document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
    document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1)
)
print("Train documents:")
pprint(train_set)


test_set = (document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)
print("\nTest documents:")
pprint(test_set)

Train documents:
(document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
 document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
 document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
 document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
 document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
 document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
 document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
 document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1))

Test documents:
(document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)


In [3]:
# Tu respuesta
from itertools import chain
from functools import partial, lru_cache
from math import log


NUM_CLASSES = len({doc.class_ for doc in train_set})

# Function to get de prior of a class
# Count the documents in a class divided by the total document count
@lru_cache()
def class_prior(documents, class_):
    return len([doc for doc in documents if doc.class_ == class_]) / len(
        documents
    )


# Function to get a word likelihood given a class
@lru_cache()
def word_likelihood(documents, word, class_):
    vocab = {word for doc in documents for word in doc.words}
    superdocument = list(
        chain.from_iterable(
            [doc.words for doc in documents if doc.class_ == class_]
        )
    )
    return (superdocument.count(word) + 1) / (len(superdocument) + len(vocab))


# Hago los calculos lazy
trained_class_prior = partial(class_prior, train_set)
trained_word_likelihood = partial(word_likelihood, train_set)


def predict(documents):
    predicted_documents = []
    for doc in documents:
        # implemented using log sum instead of multiplication
        class_probabilities = [
            log(trained_class_prior(class_))
            + sum(
                [
                    log(trained_word_likelihood(word, class_))
                    for word in doc.words
                ]
            )
            for class_ in range(NUM_CLASSES)
        ]
        predicted_documents.append(
            document(
                doc.words, class_probabilities.index(max(class_probabilities))
            )
        )
    return predicted_documents


predicted = predict(test_set)
for doc in predicted:
    print(f"La clase predicha del documento {doc.words} es: {doc.class_}")


La clase predicha del documento ('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03') es: 0


In [13]:
for word in test_set[0].words:
    print(word, trained_word_likelihood(word, 1))

w02 0.08333333333333333
w09 0.125
w06 0.125
w01 0.08333333333333333
w05 0.16666666666666666
w04 0.08333333333333333
w03 0.020833333333333332
w03 0.020833333333333332
